In [1]:
#Install packages
%pip install faiss-cpu
%pip install sentence-transformers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Step 1: Create source embeddings for the text column

In [6]:
from sentence_transformers import SentenceTransformer

/Users/ankushbhatt/Desktop/LLM_Learning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ankushbhatt/Desktop/LLM_Learning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [8]:
vectors.shape

(8, 768)

In [9]:
dim = vectors.shape[1]
dim

768

Step 2: Build a FAISS Index for vectors

In [10]:
import faiss
index = faiss.IndexFlatL2(dim)

Step 3: Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index.

In [11]:
index.add(vectors)

In [12]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x32d6aff90> >

Step 4: Encode Search Text using same encoder and normalize the output vector.

In [13]:
search_query = "I want to buy a polo t-shirt."
# search_query = "looking for places to visit during the holidays."
# search_query = "An apple a day keeps the doctor away."
vec = encoder.encode(search_query)
vec.shape

(768,)

In [14]:
import numpy as np 
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

Step 5: Search for similar vector in the FAISS index created.

In [15]:
distances, I = index.search(svec, k=2)
distances


array([[1.3669324, 1.3831215]], dtype=float32)

In [16]:
I

array([[3, 2]])

In [17]:
I.tolist()

[[3, 2]]

In [18]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [19]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [20]:
search_query

'I want to buy a polo t-shirt.'